In [1]:
import os
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.classification import GBTClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [2]:

def spark_session():
    # Stop any old session so new configs take effect in notebooks
    return (
        SparkSession.builder
        .appName("MySQL_to_Delta_on_MinIO")
        .master("spark://spark-master:7077")
        .config("spark.jars.packages",
                ",".join([
                    # Delta
                    "io.delta:delta-spark_2.12-3.1.0",
                    # MySQL JDBC
                    "mysql:mysql-connector-java:8.0.33",
                    # S3A / MinIO (versions must match your Hadoop)
                    "org.apache.hadoop:hadoop-aws:3.3.2",
                    "com.amazonaws:aws-java-sdk-bundle:1.11.1026",
                ]))
        # Delta integration
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO (S3A) configs
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        # Resources
        .config("spark.executor.cores", "2")
        .config("spark.executor.memory", "2g")
        .config("spark.executor.memoryOverhead", "512m")
        .getOrCreate()
    )

s = spark_session()


In [3]:
gold = os.getenv("GOLD_PATH","s3a://deltabucket/gold/wholeCorp_delta")
df = s.read.format("delta").load(gold)


In [5]:

# Label from 公司狀態 → Active=1 else 0 (adjust mapping as needed)
base = s.read.format("delta").load(os.getenv("SILVER_PATH","s3a://deltabucket/silver/wholeCorp_delta"))\
        .select("統一編號","公司狀態")
data = df.join(base, "統一編號", "left")
data = data.withColumn("label", F.when(F.col("公司狀態")=="核准設立", 1).otherwise(0))

# Split
train, test = data.randomSplit([0.8, 0.2], seed=42)


In [7]:
train.show()

+--------+------------------+--------------------+------------------------------------+-----+
|統一編號|          公司名稱|            features|                            公司狀態|label|
+--------+------------------+--------------------+------------------------------------+-----+
|00000228|三合益煤油有限公司|(262149,[62903,26...|                            核准設立|    1|
|00000228|三合益煤油有限公司|(262149,[62903,26...|                            核准設立|    1|
|00000696|        義達什貨店|      (262149,[],[])|                                NULL|    0|
|00000696|        義達什貨店|      (262149,[],[])|                                NULL|    0|
|00000696|        義達什貨店|      (262149,[],[])|                                NULL|    0|
|00001514|        五洲豆漿店|(262149,[15321,44...|核准設立 「查詢最新營業狀況請至財...|    0|
|00012809|            三和行|(262149,[62903,26...|核准設立 「查詢最新營業狀況請至財...|    0|
|00012809|            三和行|(262149,[62903,26...|核准設立 「查詢最新營業狀況請至財...|    0|
|00012809|            三和行|(262149,[62903,26...|核准設立 「查詢最新營業狀況請至財...|    0|
|00

In [8]:

# Model
clf = GBTClassifier(labelCol="label", featuresCol="features", maxDepth=6, maxIter=80)

paramGrid = (ParamGridBuilder()
             .addGrid(clf.maxDepth, [4,6,8])
             .addGrid(clf.maxIter, [50,100])
             .build())

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction",
                                          metricName="areaUnderPR")
cv = CrossValidator(estimator=clf, estimatorParamMaps=paramGrid, evaluator=evaluator,
                    numFolds=3, parallelism=4)

In [10]:
best = cv.fit(train)
preds = best.transform(test)
print("AUPRC:", evaluator.evaluate(preds))

# Save model path for scoring
out_uri = os.getenv("STATUS_MODEL_URI","s3a://deltabucket/models/status_gbt")
best.bestModel.write().overwrite().save(out_uri)


Py4JJavaError: An error occurred while calling o230.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: ShuffleMapStage 66 (mapPartitions at RandomForest.scala:644) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 7 partition 99
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1739)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1686)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1685)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala:1685)
	at org.apache.spark.MapOutputTrackerWorker.getMapSizesByExecut